# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Note: python would only acknowledge the existence of the weather data file if I gave it the whole file address. This has been a persistent problem for me, and I don't know how to resolve it.
weather_data_file = "C:\\Users\\pippi\\OneDrive\\Documents\\nu-chi-data-pt-06-2021-u-c-master\\06-Python-APIs\\Homework\\due_July29\\Instructions\\output_data\\weather_data.csv"
weather_data_original = pd.read_csv(weather_data_file, dtype = "object", encoding = 'utf-8')
weather_data_original.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Punta Arenas,-53.15,-70.9167,51.12,66,0,19.57,CL,1628079633
1,1,Itarema,-2.9248,-39.9167,79.81,78,75,12.55,BR,1628066751
2,2,Albany,42.6001,-73.9662,76.64,65,100,2.35,US,1628070667
3,3,Mataura,-46.1927,168.8643,44.47,70,100,5.39,NZ,1628107190
4,4,Geraldton,-28.7667,114.6,44.13,93,0,6.91,AU,1628118135


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Use the Lat and Lng as locations and Humidity as the weight.
locations = weather_data_original[["Lat", "Lng"]].astype(float)
humidity = weather_data_original['Humidity']

# Create the Heatmap
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights = humidity,
                                dissipating=False, max_intensity=100,
                                point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
# Narrow down the cities to fit weather conditions
weather_data_ideal = weather_data_original.loc[((weather_data_original['Max Temp'].astype(float) > 70) & (weather_data_original['Max Temp'].astype(float) < 80))]
                                              #(weather_data_original['Wind Speed'].astype(float) < 10.00) | (weather_data_original['Cloudiness'] == 0)
weather_data_idealer = weather_data_ideal.loc[(weather_data_ideal['Wind Speed'].astype(float) < 10.00)]

weather_data_idealest = weather_data_idealer.loc[(weather_data_idealer['Cloudiness'].astype(float) == 0)]
weather_data_idealest.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,Chapais,49.7834,-74.8492,71.38,54,0,4.36,CA,1628069647
165,165,Bayir,37.2687,28.2167,77.52,31,0,5.77,TR,1628133302
203,203,Ribeira Grande,38.5167,-28.7,72.99,89,0,6.89,PT,1628060360
228,228,Mitsamiouli,-11.3847,43.2844,74.59,73,0,1.57,KM,1628133803
245,245,Mashhad,36.297,59.6062,77.14,38,0,4.61,IR,1628125881


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
# Store into variable named hotel_df.
hotel_df = pd.DataFrame(weather_data_idealest)
print(hotel_df)

    Unnamed: 0            City       Lat       Lng Max Temp Humidity  \
8            8         Chapais   49.7834  -74.8492    71.38       54   
165        165           Bayir   37.2687   28.2167    77.52       31   
203        203  Ribeira Grande   38.5167     -28.7    72.99       89   
228        228     Mitsamiouli  -11.3847   43.2844    74.59       73   
245        245         Mashhad    36.297   59.6062    77.14       38   
271        271        Bandrele  -12.9067   45.1914    71.35       88   
299        299         Cayenne    4.9333  -52.3333    79.09       84   
414        414    Marsá Maţrūḩ   31.3525   27.2453     79.0       89   
425        425             Fes   34.0372   -4.9998    77.25       43   

    Cloudiness Wind Speed Country        Date  
8            0       4.36      CA  1628069647  
165          0       5.77      TR  1628133302  
203          0       6.89      PT  1628060360  
228          0       1.57      KM  1628133803  
245          0       4.61      IR  1628

In [14]:
# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

In [15]:
# Set parameters to search for hotels with 5000 meters.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
params = {
    "location": "",
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [16]:
# Hit the Google Places API for each city's coordinates.
for index, row in hotel_df.iterrows():
    params['location'] = f"{row['Lat']},{row['Lng']}"
    # params['keyword']
    response = requests.get(base_url, params=params).json()
    results = response['results']

# Store the first Hotel result into the DataFrame.
    try:
        print(f"The first hotel found in {row['City']} is {results[0]['name']}.")
              
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except(KeyError, IndexError):
        print("------------------")

The first hotel found in Chapais is Hôtel Opémiska.
The first hotel found in Bayir is Hotel Babadag.
The first hotel found in Ribeira Grande is Quinta da Meia Eira.
The first hotel found in Mitsamiouli is Foyer ADM.
The first hotel found in Mashhad is Khanye Sabz Hotel.
The first hotel found in Bandrele is Les Baobabs.
The first hotel found in Cayenne is Hôtel Le Dronmi.
The first hotel found in Marsá Maţrūḩ is Beau Site.
The first hotel found in Fes is Hotel Mounia Fès.


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]].astype(float)

In [18]:
# Add marker layer on top of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))